# ETL I

## Task I - Define correct schema for json data

* load json dataset
* look at the infered schema (is it inferred correctly or is it wrong?)
* define the schema explicitly
* see what happens if the schema is defined wrong

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, explode, split, regexp_replace, collect_list

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

import os

In [2]:
spark = (
    SparkSession
    .builder
    .appName('ETL I')
    .getOrCreate()
)

In [3]:
print(spark.version)

3.0.0


The input dataset is in the json format and is in the `data/questios-json` folder. Below is the path definition:

In [4]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

data_input_path = os.path.join(project_path, 'data/questions-json')

output_path = os.path.join(project_path, 'output/questions-transformed')

#### First let Spark infer the schema:

In [5]:
questionsDF = (
    spark
    .read
    .format('json')
    .option('path', data_input_path)
    .load()
)

Note: Here we have only 8 json files. In case where you have lots of json files and you know that each file has the same schema, consider loading only one file to check the schema. Inferring the schema from many files can be expensive.

In [6]:
questionsDF.printSchema()

root
 |-- accepted_answer_id: long (nullable = true)
 |-- answers: long (nullable = true)
 |-- body: string (nullable = true)
 |-- comments: long (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- question_id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- tags: string (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- views: long (nullable = true)



#### Define the schema:

Here it depends on the Spark version. 
* If Spark version < 3.0, the data type of `creation_date` is inferred as `StringType` however in reality it is a Timestamp. Define the schema by hand and provide it to create the DataFrame
* If Spark version >= 3.0, the data type of `creation_date` is inferred correctly 

In [7]:
# You can skip this if using Spark 3.x, because the schema is inferred correctly

json_schema = StructType(
    [
        StructField('question_id', LongType(), True),
        StructField('creation_date', TimestampType(), True),
        StructField('title', StringType(), True),
        StructField('body', StringType(), True),
        StructField('tags', StringType(), True),
        StructField('accepted_answer_id', LongType(), True),
        StructField('answers', LongType(), True),
        StructField('comments', LongType(), True),
        StructField('user_id', LongType(), True),
        StructField('views', LongType(), True),
    ]
)

questionsDF = (
    spark
    .read
    .schema(json_schema)
    .format('json')
    .option('path', data_input_path)    
    .load()
)

In [8]:
questionsDF.show(truncate=10, n=5)

+-----------+-------------+----------+----------+----------+------------------+-------+--------+-------+-----+
|question_id|creation_date|     title|      body|      tags|accepted_answer_id|answers|comments|user_id|views|
+-----------+-------------+----------+----------+----------+------------------+-------+--------+-------+-----+
|   61416257|   2020-04...|Ag-Grid...|<p>Ag-g...| <ag-grid>|          61420331|      1|       1|4292512|   21|
|   61482176|   2020-04...|Optiona...|<p>My c...|<c#><fu...|              null|      0|       4|3603502|   28|
|   61919808|   2020-05...|Matchin...|<p>Ther...|<python...|              null|      0|       3|4453105|   40|
|   60340057|   2020-02...|Knockou...|<p>I'm ...|<knocko...|          60340749|      1|       0|3157885|   35|
|   62001217|   2020-05...|Python ...|<p>I ru...|<python...|              null|      1|       0|4220475|   17|
+-----------+-------------+----------+----------+----------+------------------+-------+--------+-------+-----+
o

In [9]:
questionsDF.count()

195179

#### What happens if the actual data type doesn't match the schema:

* set `title` as `LongType` in the defined schema

Hint
* Different things will happen depending on the `mode` option, where `mode` is one of the following:
    * FAILFAST
    * DROPMALFORMED
    * PERMISSIVE (default)


In [10]:
# Define the schema with a mistake in the title column:

wrong_schema = StructType(
    [
        StructField('question_id', LongType(), True),
        StructField('creation_date', TimestampType(), True),
        StructField('title', LongType(), True),
        StructField('body', StringType(), True),
        StructField('tags', StringType(), True),
        StructField('accepted_answer_id', LongType(), True),
        StructField('answers', LongType(), True),
        StructField('comments', LongType(), True),
        StructField('user_id', LongType(), True),
        StructField('views', LongType(), True),
    ]
)

In [11]:
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'PERMISSIVE') # this is the default
    .option('path', data_input_path)    
    .load()
).show(truncate=15, n=5)

+-----------+---------------+-----+---------------+---------------+------------------+-------+--------+-------+-----+
|question_id|  creation_date|title|           body|           tags|accepted_answer_id|answers|comments|user_id|views|
+-----------+---------------+-----+---------------+---------------+------------------+-------+--------+-------+-----+
|   61416257|2020-04-24 2...| null|<p>Ag-grid's...|      <ag-grid>|          61420331|      1|       1|4292512|   21|
|   61482176|2020-04-28 1...| null|<p>My curren...|<c#><functio...|              null|      0|       4|3603502|   28|
|   61919808|2020-05-20 2...| null|<p>There are...|<python><reg...|              null|      0|       3|4453105|   40|
|   60340057|2020-02-21 1...| null|<p>I'm hopin...|<knockout.js...|          60340749|      1|       0|3157885|   35|
|   62001217|2020-05-25 1...| null|<p>I run a p...|<python><mys...|              null|      1|       0|4220475|   17|
+-----------+---------------+-----+---------------+-----

In [12]:
# shows no records
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'DROPMALFORMED')
    .option('path', data_input_path)    
    .load()
).show(truncate=15, n=5)

+-----------+-------------+-----+----+----+------------------+-------+--------+-------+-----+
|question_id|creation_date|title|body|tags|accepted_answer_id|answers|comments|user_id|views|
+-----------+-------------+-----+----+----+------------------+-------+--------+-------+-----+
+-----------+-------------+-----+----+----+------------------+-------+--------+-------+-----+



In [13]:
# throws an error
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'FAILFAST')
    .option('path', data_input_path)    
    .load()
)#.show(truncate=15, n=5)

DataFrame[question_id: bigint, creation_date: timestamp, title: bigint, body: string, tags: string, accepted_answer_id: bigint, answers: bigint, comments: bigint, user_id: bigint, views: bigint]

#### Note

* To read more about schema inferrence and schema evolution of json files in Spark SQL, read my article: https://medium.com/swlh/notes-about-json-schema-handling-in-spark-sql-be1e7f13839d


## Task II - Transfrom json to parquet and convert String column to an array

* convert column tags to array of tags 
* &lt;tag1&gt;&lt;tag2&gt;&lt;tag3&gt; ---> [tag1, tag2, tag3]

#### Convert tags to an array

Hint
* use [split](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split) to get an array
* [explode](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode) the array to access each element separately
* use [regexp_replace](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_replace) and split on ><
* [groupBy](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) + [collect_list](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.collect_list)
* join with original questions DataFrame

In [14]:
resultDF = (
    questionsDF
    .withColumn('tags_arr', split('tags', '><'))
    .withColumn('tag', explode('tags_arr'))
    .withColumn('tag', regexp_replace('tag', '(<|>)', ''))
    .groupBy('question_id')
    .agg(collect_list('tag').alias('tags'))
    .join(questionsDF.drop('tags'), 'question_id')
)

#### Note
This is an old-school solution used rather before 2.4. Since 2.4 we have higher order functions that can solve the problem more elegantly (we will see that later in the section with Higher Order Functions)

There are also some side-effects of this solution:

1. groupBy creates a shuffle (quite expensive)
2. the elements in the final array may come in different order
3. the groupBy key must be unique, otherwise we will reduce it

#### Save the data

Hint:
* use [write](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.write) + [save](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter.save)
* [repartition](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition) the data to 8 partitions before saving
 * this will create 8 files
 
Note
* there are also other options how to save data with Spark and we will cover them in the Tables notebook

In [15]:
(
    resultDF
    .repartition(8)
    .write
    .mode('overwrite')
    .option('path', output_path)
    .save()
)

<b>Check if we saved the data correctly:</b>

In [16]:
checkDF = (
    spark
    .read
    .parquet(output_path)
)

In [17]:
checkDF.count()

195179

In [18]:
checkDF.show(truncate=10, n=5)

+-----------+----------+-------------+----------+----------+------------------+-------+--------+-------+-----+
|question_id|      tags|creation_date|     title|      body|accepted_answer_id|answers|comments|user_id|views|
+-----------+----------+-------------+----------+----------+------------------+-------+--------+-------+-----+
|   44038456|[apache...|   2017-05...|How to ...|<p>I ha...|              null|      8|       1|6214294|20288|
|   35059996|[html, ...|   2016-01...|Skip ht...|<p>I ha...|          35061376|      2|       0| 847172|  126|
|    6040824|     [wpf]|   2011-05...|Applica...|<p>I wr...|              null|      1|       1| 758659|  102|
|   15358973|[ruby-o...|   2013-03...|Rails S...|<p>I am...|              null|      1|       0|1108988| 1051|
|   39132750|[androi...|   2016-08...|Android...|<p>I ha...|          39152635|      3|       2|5889375|  632|
+-----------+----------+-------------+----------+----------+------------------+-------+--------+-------+-----+
o

In [19]:
checkDF.select('tags').show(truncate=False, n=10)

+-----------------------------------------------------+
|tags                                                 |
+-----------------------------------------------------+
|[apache, reactjs, webpack]                           |
|[html, twig, twig-extension]                         |
|[wpf]                                                |
|[ruby-on-rails-3, http, ssl, savon]                  |
|[android, image, background-image, responsive-images]|
|[c#, c++, visual-studio-2013, deployment, windows-ce]|
|[asp.net-core, odata]                                |
|[javascript, google-cloud-firestore]                 |
|[android, android-service, battery, batterylevel]    |
|[java, swing]                                        |
+-----------------------------------------------------+
only showing top 10 rows



In [20]:
checkDF.printSchema()

root
 |-- question_id: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- creation_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- accepted_answer_id: long (nullable = true)
 |-- answers: long (nullable = true)
 |-- comments: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- views: long (nullable = true)



In [21]:
spark.stop()